# 量子回路最適化

例えば、Hゲートを同じビットに連続して作用させるとゲート操作をしていないのと恒等な量子状態が得られます。  
<img src="fig/circ.png" width="500">

量子回路が深くなる（ゲート数が多くなる）ほど、誤差が大きくなるNISQ デバイスでは左辺のような状況は除去したい。  

その他にも下記のような恒等な関係があります。  
<img src="fig/relation.png" width="700">

`PauliSimp`関数や`get_compiled_circuit`の紹介を行ったが、`TKET`にはその他にも量子回路を最適化するための多くの機能が備わっている。

### 1. TKETの量子回路の最適化
`TKET`の最適化の機能は次のように２つの種類に分けられる

a. デバイス非依存型：量子ゲートの総数を減らすことを目的とした量子回路の最適化:
    `FullPeepholeOptimise` 
    `PauliSimp` 
    `PauliSquash` 
   
b. デバイス依存型：量子デバイスの設計の設計を考慮した量子回路の最適化:
    `get_compiled_circuit`

### 2. いくつかの量子回路の最適化関数や変換
a. `CliffordSimp`,

b. `RemoveRedundancies`, 

c. `CommuteThroughMultis`, 

d. `EulerAngleReduction`,

### 3. 量子回路の最適化のカスタマイズ (オプション)
- `SequencePass`

について紹介する。

### 参照
- [pytket ドキュメント](https://tket.quantinuum.com/api-docs/)
- [pytket ユーザーガイド](https://tket.quantinuum.com/user-guide/)


## 0. 量子回路の準備
ランダムなゲートの選択によって量子回路を生成し、回路の深さとCXの深さを数える。

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import OpType
from pytket.pauli import Pauli
from pytket.circuit import PauliExpBox, fresh_symbol, OpType
from pytket.passes import DecomposeBoxes
#box = PauliExpBox([Pauli.I, Pauli.Z, Pauli.X, Pauli.Y], fresh_symbol('tm'))
#from pytket.utils import Graph
import numpy as np

In [ ]:
def random_circuit(
    n_qubits, depth, max_ops=3, measure=False, seed=None
):
    if max_ops < 1 or max_ops > 3:
        raise print("max_ops must be between 1 and 3")
    op1types = [OpType.X, OpType.Y, OpType.Z, OpType.H, OpType.S, OpType.Sdg, OpType.T, OpType.Tdg, OpType.V, OpType.Vdg]
    op2types = [OpType.CX, OpType.CY, OpType.CZ, OpType.CV, OpType.CVdg, OpType.CH]
    op3types = [OpType.CCX]
    circ = Circuit(n_qubits)
    if seed is None:
        rng = np.random.default_rng()
    else:
        rng = np.random.default_rng(seed)
    for _ in range(depth):
        choose_qubits = list(range(n_qubits))
        rng.shuffle(choose_qubits)
        while choose_qubits:
            max_possible = min(len(choose_qubits), max_ops)
            n_ops = rng.choice(range(max_possible)) + 1
            ops = [choose_qubits.pop() for _ in range(n_ops)]
            if n_ops == 1:
                ope = rng.choice(op1types)
            elif n_ops == 2:
                ope = rng.choice(op2types)
            elif n_ops == 3:
                ope = rng.choice(op3types)
            circ.add_gate(ope, ops)
    if measure:
        circ.measure_all()
    return circ

def random_circuit_para(
    n_qubits, depth, max_ops=3, measure=False, seed=None
):
    if max_ops < 1 or max_ops > 3:
        raise print("max_ops must be between 1 and 3")
    op1types = [OpType.X, OpType.Y, OpType.Z, OpType.H, OpType.S, OpType.Sdg, OpType.T, OpType.Tdg, OpType.V, OpType.Vdg, OpType.Rx, OpType.Ry, OpType.Rz]
    op2types = [OpType.XXPhase, OpType.YYPhase, OpType.ZZPhase, OpType.CX, OpType.CY, OpType.CZ, OpType.CV, OpType.CVdg, OpType.CH, OpType.CRx, OpType.CRy, OpType.CRz]
    op3types = [OpType.CCX]
    para1 = [OpType.Rx, OpType.Ry, OpType.Rz, OpType.CRx, OpType.CRy, OpType.CRz, OpType.XXPhase, OpType.YYPhase, OpType.ZZPhase]
    circ = Circuit(n_qubits)
    if seed is None:
        rng = np.random.default_rng()
    else:
        rng = np.random.default_rng(seed)
    for _ in range(depth):
        choose_qubits = list(range(n_qubits))
        rng.shuffle(choose_qubits)
        while choose_qubits:
            max_possible = min(len(choose_qubits), max_ops)
            n_ops = rng.choice(range(max_possible)) + 1
            ops = [choose_qubits.pop() for _ in range(n_ops)]
            if n_ops == 1:
                ope = rng.choice(op1types)
            elif n_ops == 2:
                ope = rng.choice(op2types)
            elif n_ops == 3:
                ope = rng.choice(op3types)
            if ope in para1:
                circ.add_gate(ope, rng.uniform(0, 2 * np.pi), ops)
            else:
                circ.add_gate(ope, ops)
    if measure:
        circ.measure_all()
    return circ

def get_random_pauli_gadgets(n_qubits, n_pauli_gadgets, max_entangle):
    paulis = [Pauli.I, Pauli.X, Pauli.Y, Pauli.Z]
    circ = Circuit(n_qubits)
    for i in range(n_pauli_gadgets):
        ls_paulis = [np.random.choice(paulis) for k in range(max_entangle)]
        if ls_paulis.count(Pauli.Y) % 2 == 0:
            continue
        if len(ls_paulis) - ls_paulis.count(Pauli.I) <= 1:
            continue
        qubits = np.random.choice(
            [i for i in range(n_qubits)], size=max_entangle, replace=False
        )
        box = PauliExpBox(ls_paulis, fresh_symbol('a'))
        circ.add_pauliexpbox(box, sorted(qubits))
    DecomposeBoxes().apply(circ)
    return circ

#### 深さ200のランダムな量子回路を生成

In [ ]:
circ1 = random_circuit(n_qubits=2, depth=200, max_ops=1)
circ2 = random_circuit(n_qubits=3, depth=200, max_ops=2)
circ3 = random_circuit(n_qubits=4, depth=200, max_ops=3)
print('Circuit depth: ', circ1.depth())
print('Control depth: ', circ1.depth_by_type({OpType.CX, OpType.CY, OpType.CZ, OpType.CV, OpType.CVdg, OpType.CH}))
render_circuit_jupyter(circ1)
print('Circuit depth: ', circ2.depth())
print('Control depth: ', circ2.depth_by_type({OpType.CX, OpType.CY, OpType.CZ, OpType.CV, OpType.CVdg, OpType.CH}))
render_circuit_jupyter(circ2)
print('Circuit depth: ', circ3.depth())
print('Control depth: ', circ3.depth_by_type({OpType.CX, OpType.CY, OpType.CZ, OpType.CV, OpType.CVdg, OpType.CH}))
print('CCX depth: ', circ3.depth_by_type(OpType.CCX))
render_circuit_jupyter(circ3)

## 1. TKETの量子回路の最適化

#### a1. 量子ゲートの総数を減らすことを目的とした量子回路の最適化(量子デバイス非依存): `FullPeepholeOptimise`
`FullPeepholeOptimise`は`CliffordSimp`, `RemoveRedundancies`, `CommuteThroughMultis`, `KAKDecomposition`, `EulerAngleReduction`から構成されている量子回路最適化です。
こちらは、量子デバイスが用意しているゲートセットやアーキテクチャー等を考慮せずに最適化されます。

`PauliSquash` 関数を利用した時と同様にして、`FullPeepholeOptimise`を利用することができます。

In [ ]:
from pytket.passes import FullPeepholeOptimise
#optimization of circ1
circ1x=circ1.copy()
FullPeepholeOptimise().apply(circ1x)
print('-Optimized')
print('Circuit depth: ', circ1x.depth())
print('CX depth: ', circ1x.depth_by_type(OpType.CX))
print('-Unotimized')
print('Circuit depth: ', circ1.depth())
print('CX depth: ', circ1.depth_by_type(OpType.CX))
render_circuit_jupyter(circ1x)
#optimization of circ2
circ2x=circ2.copy()
FullPeepholeOptimise().apply(circ2x)
print('-Optimized')
print('Circuit depth: ', circ2x.depth())
print('CX depth: ', circ2x.depth_by_type(OpType.CX))
print('-Unotimized')
print('Circuit depth: ', circ2.depth())
print('CX depth: ', circ2.depth_by_type(OpType.CX))
render_circuit_jupyter(circ2x)
#optimization of circ3
circ3x=circ3.copy()
FullPeepholeOptimise().apply(circ3x)
print('-Optimized')
print('Circuit depth: ', circ3x.depth())
print('CX depth: ', circ3x.depth_by_type(OpType.CX))
print('CCX depth: ', circ3x.depth_by_type(OpType.CCX))
print('-Unotimized')
print('Circuit depth: ', circ3.depth())
print('CX depth: ', circ3.depth_by_type(OpType.CX))
print('CCX depth: ', circ3.depth_by_type(OpType.CCX))
render_circuit_jupyter(circ3x)

ここで、$\mathrm{TK1}(\alpha,\beta,\gamma)$はオイラー角
$$
Rz(\alpha)Rx(\beta)Rz(\gamma)
$$
を表す

In [ ]:
circ = Circuit(3).CCX(0,1,2)
FullPeepholeOptimise().apply(circ)
render_circuit_jupyter(circ)

#### a2. 量子ゲートの総数を減らすことを目的とした量子回路の最適化: `PauliSquash`
`PauliSquash`=`PauliSimp`+`FullPeepholeOptimise`

In [ ]:
circg = get_random_pauli_gadgets(n_qubits=4, n_pauli_gadgets=100, max_entangle=3)
print('Circuit depth: ', circg.depth())
print('CX depth: ', circg.depth_by_type(OpType.CX))
render_circuit_jupyter(circg)

In [ ]:
from pytket.passes import PauliSimp, PauliSquash
circgSi = circg.copy()
circgSq = circg.copy()
PauliSimp().apply(circgSi)
print('Circuit depth: ', circgSi.depth())
print('CX depth: ', circgSi.depth_by_type(OpType.CX))
render_circuit_jupyter(circgSi)
PauliSquash().apply(circgSq)
print('Circuit depth: ', circgSq.depth())
print('CX depth: ', circgSq.depth_by_type(OpType.CX))
render_circuit_jupyter(circgSq)

#### b. 量子デバイスの設計を考慮した量子回路の最適化(量子デバイス依存): `get_compiled_circuit`
'TKET'では量子デバイス非依存の量子回路の最適化だけでなく、量子デバイスのアーキテクチャに対して量子回路を最適化する関数が用意されてます。  
こちらは、量子デバイスが用意しているゲートセットやアーキテクチャー等を考慮した量子回路の最適化です。

Optimisationのレベルを0,1,2から選択でき、それぞれ以下のようになっています。

|  レベル |  概要  |
| :---- | :---- |
|  0  | Just solves the constraints as simply as possible. No optimisation. |
|  1  | Adds basic optimisations (those covered by the SynthesiseX() passes) for efficient compilation. |
|  2  (default)| Extends to more intensive optimisations (those covered by the FullPeepholeOptimise() pass). |

##### IBMのクラウドシミュレータ（ibm_pittsburgh）にジョブを実行
<img src="./fig/ibm_pittsburgh.png" width="250">  

IBM Quantum device `ibm_pittsburgh`  
https://quantum.ibm.com/services/resources?tab=systems&system=ibm_pittsburgh



##### IBM Quantum トークンの設定

In [ ]:
# Replace the placeholders with your actual values
ibm_token = '<your_ibm_token_here>'
# If you need a specific instance set it here, otherwise you can omit this line
# inst = '<your_instance_CRN_here>'

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
# IBM QuantumのAPIキーを設定
QiskitRuntimeService.save_account(channel="ibm_quantum_platform", token=ibm_token, overwrite=True)

In [ ]:
from pytket.extensions.qiskit import IBMQBackend, IBMQEmulatorBackend

In [ ]:
device = IBMQBackend.available_devices()
[dev.device_name for dev in device]

In [ ]:
#シミュレータを指定したのと全く同じように量子コンピュータを指定
# ibm_backend = AerBackend()
#実機にジョブを投げる
#ibm_backend = IBMQBackend("ibm_aachen")
#実機のノイズを取り出し、ノイズモデルを作成
ibm_backend = IBMQEmulatorBackend("ibm_aachen")

In [ ]:
print('Original')
print('Circuit depth: ', circ3x.depth())
print('CX depth: ', circ3x.depth_by_type(OpType.CX))
circ3x0 = circ3x.copy()
circ3x0 = ibm_backend.get_compiled_circuit(circ3x0,optimisation_level=0)
print('Level 0')
print('Circuit depth: ', circ3x0.depth())
print('CX depth: ', circ3x0.depth_by_type(OpType.CX))
print('ECR depth: ', circ3x0.depth_by_type(OpType.ECR))
circ3x1 = circ3x.copy()
circ3x1 = ibm_backend.get_compiled_circuit(circ3x1,optimisation_level=1)
print('Level 1')
print('Circuit depth: ', circ3x1.depth())
print('CX depth: ', circ3x1.depth_by_type(OpType.CX))
print('ECR depth: ', circ3x1.depth_by_type(OpType.ECR))
circ3x2 = circ3x.copy()
circ3x2 = ibm_backend.get_compiled_circuit(circ3x2,optimisation_level=2)
print('Level 2')
print('Circuit depth: ', circ3x2.depth())
print('CX depth: ', circ3x2.depth_by_type(OpType.CX))
print('ECR depth: ', circ3x2.depth_by_type(OpType.ECR))

In [ ]:
print('Original')
render_circuit_jupyter(circ3x)
print('Level 0')
render_circuit_jupyter(circ3x0)
print('Level 1')
render_circuit_jupyter(circ3x1)
print('Level 2')
render_circuit_jupyter(circ3x2)

## 2. いくつかの量子回路の最適化関数や変換
a. `CliffordSimp`

b. `RemoveRedundancies`

c. `CommuteThroughMultis`

e. `EulerAngleReduction`

### a. `CliffordSimp`
2量子ビットゲートの数を減らすことができるクリフォードゲートの特定のシーケンスを探す。

In [ ]:
# Circuit optimization by using compiler passes.
from pytket.passes import CliffordSimp
# A basic inefficient pattern can be reduced by 1 CX
simple = Circuit(2)
simple.CX(0, 1).S(1).CX(1, 0)
render_circuit_jupyter(simple)

CliffordSimp().apply(simple)
render_circuit_jupyter(simple)

In [ ]:
from pytket import Circuit, OpType
from pytket.passes import CliffordSimp

# The same pattern, up to commutation and local Clifford algebra
circ = Circuit(3)
circ.Rz(0.2, 0)
circ.Rx(0.35, 1)
circ.V(0).H(1).CX(0, 1).CX(1, 2).Rz(-0.6, 2).CX(1, 2).CX(0, 1).Vdg(0).H(1)
circ.H(1).H(2).CX(0, 1).CX(1, 2).Rz(0.8, 2).CX(1, 2).CX(0, 1).H(1).H(2)
circ.Rx(0.1, 1)
print('Circuit depth: ', circ.depth())
print('CX depth: ', circ.depth_by_type(OpType.CX))
render_circuit_jupyter(circ)
CliffordSimp().apply(circ)
print('Circuit depth: ', circ.depth())
print('CX depth: ', circ.depth_by_type(OpType.CX))
render_circuit_jupyter(circ)

### b. `RemoveRedundancies`
ゼロパラメータの回転ゲート、ゲートと逆のペア、同じ基底で隣接する回転ゲートなどを削減します。

In [ ]:
from pytket.passes import RemoveRedundancies
circ = Circuit(3, 3)
circ.Rx(0.92, 0).CX(1, 2).Rx(-0.18, 0)  # Adjacent Rx gates can be merged
circ.CZ(0, 1).Ry(0.11, 2).CZ(0, 1)      # CZ is self-inverse
circ.add_gate(OpType.XXPhase, 0.6, [0, 1])
circ.add_gate(OpType.YYPhase, 0, [0, 1])    # 0-angle rotation does nothing
circ.add_gate(OpType.ZZPhase, -0.84, [0, 1])
circ.Rx(0.03, 0).Rz(-0.9, 1).measure_all()  # Effect of Rz is eliminated by measurement
print('Circuit depth: ', circ.depth())
print('CX depth: ', circ.depth_by_type(OpType.CX))
render_circuit_jupyter(circ)

RemoveRedundancies().apply(circ)
print('Circuit depth: ', circ.depth())
print('CX depth: ', circ.depth_by_type(OpType.CX))
render_circuit_jupyter(circ)


### c. `CommuteThroughMultis`
"1量子ビットゲートと可換な"2量子ビットゲートが先に作用している場合、これらを交換する。

In [ ]:
from pytket.passes import CommuteThroughMultis
circD = Circuit(3)
circD.CX(0,2).X(2)
render_circuit_jupyter(circD)
CommuteThroughMultis().apply(circD)
render_circuit_jupyter(circD)

### d. `EulerAngleReduction`
`EulerAngleReduction`では、ユニタリ行列のオイラー角分解を与える。

In [ ]:
from pytket.circuit import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter
from pytket.passes import EulerAngleReduction, AutoRebase
rebase = AutoRebase({OpType.CZ, OpType.Rx, OpType.Rz})
circE = Circuit(1)
circE.H(0).T(0)
render_circuit_jupyter(circE)
rebase.apply(circE)
render_circuit_jupyter(circE)
EulerAngleReduction(OpType.Rx, OpType.Rz).apply(circE)
render_circuit_jupyter(circE)
EulerAngleReduction(OpType.Rz, OpType.Rx).apply(circE)
render_circuit_jupyter(circE)

In [ ]:
from pytket.circuit import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter
from pytket.passes import EulerAngleReduction, AutoRebase
rebase = AutoRebase({OpType.CX, OpType.Rx, OpType.Ry})
circE = Circuit(1)
circE.H(0).T(0)
render_circuit_jupyter(circE)
rebase.apply(circE)
render_circuit_jupyter(circE)
#EulerAngleReduction(OpType.Rx, OpType.Ry).apply(circE)
#render_circuit_jupyter(circE)
EulerAngleReduction(OpType.Ry, OpType.Rx).apply(circE)
render_circuit_jupyter(circE)

## 3. 量子回路の最適化のカスタマイズ
- `SequencePass`

'TKET'にはそのほかにも多くの量子回路最適化を用意している。
これらを最適化のパスに組み込みたい場合や自分で実装した最適化関数をパスに組み込みたい場合には`SequencePass`を利用すれば、カスタマイズが可能である。

In [ ]:
from pytket import Circuit, OpType
from pytket.passes import AutoRebase, EulerAngleReduction, SequencePass
rebase = AutoRebase({OpType.CZ, OpType.Rz, OpType.Rx})
circ = Circuit(3)
circ.CX(0, 1).Rx(0.3, 1).CX(2, 1).Rz(0.8, 1)
render_circuit_jupyter(circ)

circ1 = circ.copy()
rebase.apply(circ1)
EulerAngleReduction(OpType.Rz, OpType.Rx).apply(circ1)
render_circuit_jupyter(circ1)

circ2 = circ.copy()
comp = SequencePass([rebase, EulerAngleReduction(OpType.Rz, OpType.Rx)])
comp.apply(circ2)
render_circuit_jupyter(circ2)

## 参照 

- pytket.passes https://tket.quantinuum.com/api-docs/passes.html

弊社Quantinuumのご紹介
- Quantinuum ウェブサイト（ 英語 ）： https://www.quantinuum.com/
- Quantinuum K.K. ウェブサイト（ 日本語 ）： https://quantinuum.co.jp/
- 各種技術詳細（ 英語 ）： https://www.quantinuum.com/products
- ニュース（ 日本語 ）： https://quantinuum.co.jp/news/  
- X（ 日本語 ）： https://twitter.com/quantinuum_jp?lang=en
- Quantinuum K.K.主催の勉強会（ 日本語 ）： https://quantinuum.connpass.com/  
- 採用情報（ 英語 ）：https://www.quantinuum.com/careers
- TKET slack channel：[TKET slack channel](https://join.slack.com/t/tketusers/shared_invite/zt-2aoan2s87-WDdZQeY2dbJQgAQE6O~3qg)

<img src="./fig/slack-qr.png" width="250">

